
# Reconhecimento de cores com Redes Neurais Convolucionais
#### Deep Learning Book (www.deeplearningbook.com.br) Capítulo 47

## Introdução

Problema: Dada uma imagem, definir qual a cor pertence a classe especifica. 

Usaremos, portanto, uma abordagem de aprendizagem supervisionada, onde apresentaremos ao algoritmo diversas imagens, devidamente marcadas com suas cores e então treinaremos o algoritmo. Ao final do treinamento, teremos um modelo que poderá receber novas imagens (desta vez não marcadas previamente) e então o modelo deverá ser capaz de classificar qual a cor pertencente aquela classe.

Para essa tarefa, usaremos uma arquitetura de Rede Neural Convolucional. Essa arquitetura usa métodos de convolução para poder prever características específicas de uma imagem de acordo com o que aprende em um conjunto de treinamento.

## Definição dos Dados

Temoss os seguintes dados:

Conjunto de dados de treino:

Conjunto de dados de validação:

Conjunto de dados de teste:

Usamos os dados de treino para treinar o algoritmo e então criar o modelo preditivo. Usamos os dados de validação, para avaliar o modelo durante o treinamento. Usamos os dados de teste para validar a performance do modelo já treinado, ou seja, apresentamos ao modelo dados que ele não viu durante o treinamento, a fim de garantir que ele é capaz de fazer previsões.

## Carregando os Dados

In [ ]:
from IPython.display import Image
Image(filename = 'dataset_treino/classe1/img1.jpg')

In [ ]:
Image(filename = 'dataset_treino/classe2/img1.jpg')

In [ ]:
Image(filename = 'dataset_treino/classe3/img1.jpg')

In [ ]:
Image(filename = 'dataset_treino/classe4/img1.jpg')

In [ ]:
Image(filename = 'dataset_treino/classe5/img1.jpg')

## Construindo a Rede Neural Convolucional

Nossa rede é uma sequência de camadas e podemos usar o modelo sequencial oferecido pelo Keras, que possui as funções necessárias para construir cada camada de uma rede neural convolucional.

O primeiro passo é carregar os pacotes necessários, o que é feito nas células abaixo.

O Keras utiliza o TensorFlow como backend, pois na prática o Keras é apenas uma biblioteca para simplificar a complexidade do TensorFlow. Aqui estão as versões utilizadas:

In [ ]:
import tensorflow as tf
print("Versão do TensorFlow:", tf.__version__)

In [ ]:
import keras as K
print("Versão do Keras:", K.__version__)

In [ ]:
# Imports
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense

Nós então inicializamos a nossa rede:

In [ ]:
# Inicializando a Rede Neural Convolucional
classifier = Sequential()

Agora nós definimos os parâmetros para o shape dos dados de entrada e a função de ativação. Usaremos 32 features para um array 2D e definiremos nosso array como o formato 3x3.

Converteremos todas as nossas imagens 64x64 pixels em um array 3D (pois as imagens são coloridas com 3 canais de cores).

In [ ]:
# Passo 1 - Primeira Camada de Convolução
classifier.add(Conv2D(32, (3, 3), input_shape = (64, 64, 3), activation = 'relu'))

Agora aplicamos o agrupamento (pooling) para reduzir o tamanho do mapa de features resultado da primeira camada de convolução (dividido por 2):

In [ ]:
# Passo 2 - Pooling
classifier.add(MaxPooling2D(pool_size = (2, 2)))

Adicionamos então a Segunda Camada de Convolução, tornando nossa rede um pouco mais profunda:

In [ ]:
# Adicionando a Segunda Camada de Convolução
classifier.add(Conv2D(32, (3, 3), activation = 'relu'))

Mais uma vez, aplicamos a camada de pooling à saída da camada de convolução anterior.

In [ ]:
classifier.add(MaxPooling2D(pool_size = (2, 2)))

Agora aplicamos o "achatamento" ou apenas Flatten para converter a estrutura de dados 2D resultado da camada anterior em uma estrutura 1D, ou seja, um vetor.

In [ ]:
# Passo 3 - Flattening
classifier.add(Flatten())

No próximo passo conectamos todas as camadas. Usamos uma função de ativação retificadora (relu) e então uma função de ativação sigmóide para obter as probabilidades de cada imagem conter a cor de interesse. O modelo raramente terá 100% de certeza e o que ele gera como um resultado é uma probabilidade.

In [ ]:
# Passo 4 - Full connection
classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dense(units = 1, activation = 'sigmoid'))

Finalmente nós compilamos nossa rede neural. Para compilar a rede, usamos o otimizador "Adam", um excelente algoritmo de primeira ordem para otimização baseada em gradiente de funções objetivas estocásticas, que toma como base uma estimativa adaptada de momentos de baixa ordem.

Usamos uma função log loss com "entropia binária cruzada", pois ela funciona bem com funções sigmóides. Nossa métrica será a acurácia, pois essa é nossa maior preocupação no treinamento deste tipo de modelo.

In [ ]:
# Compilando a rede
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

Neste ponto temos nossa rede construída. Precisamos agora treiná-la.

## Treinando a Rede Neural Convolucional

### Pré-Processamento

Com a rede criada, precisamos agora realizar o treinamento. Antes, porém, precisamos fazer algum pré-processamento nos dados, em nosso caso as imagens. Para essa tarefa, vamos usar a função ImageDataGenerator() do Keras e ajustar escala e zoom das imagens de treino e a escala das imagens de validação.

O pré-processamento dos dados é etapa crucial em qualquer projeto de Machine Learning e muitas técnicas podem ser usadas, sempre de acordo com os dados em mãos e o problema que estamos tentando resolver. Nos cursos da Data Science Academy ensinamos aos alunos uma variedade de técnicas.

In [ ]:
# Criando os objetos train_datagen e validation_datagen com as regras de pré-processamento das imagens
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

validation_datagen = ImageDataGenerator(rescale = 1./255)


Aplicamos então os dois objetos criados anteriormente para pré-processar os dados de treino e de validação. Lembre-se: o tratamento aplicado aos dados de validação deve ser o mesmo tratamento aplicado aos dados de treino.

In [ ]:
# Pré-processamento das imagens de treino e validação
training_set = train_datagen.flow_from_directory('dataset_treino',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

validation_set = validation_datagen.flow_from_directory('dataset_validation',
                                                        target_size = (64, 64),
                                                        batch_size = 32,
                                                        class_mode = 'binary')

### Treinamento

Usaremos 8000 passos em nosso conjunto de treinamento para cada época. Escolhemos 2000 etapas de validação para as imagens de validação.

In [ ]:
# Executando o treinamento (esse processo pode levar bastante tempo, dependendo do seu computador)
classifier.fit_generator(training_set,
                         steps_per_epoch = 8000,
                         epochs = 5,
                         validation_data = validation_set,
                         validation_steps = 2000)

Observe que ao final de cada época a acurácia aumenta, ou seja, nosso modelo está aprendendo e cometendo cada vez menos erros.

## Fazendo Previsões

In [ ]:
# Primeira Imagem
import numpy as np
from keras.preprocessing import image

test_image = image.load_img('dataset_teste/1.jpg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = classifier.predict(test_image)
training_set.class_indices

if result[0][0] == 1:
    prediction = 'Cor 1'
if result[0][0] == 2:
    prediction = 'Cor 2'
if result[0][0] == 3:
    prediction = 'Cor 3'
if result[0][0] == 4:
    prediction = 'Cor 4'
else:
    prediction = 'Cor 5'

Image(filename='dataset_teste/1.jpg')

In [ ]:
# Previsão da primeira imagem
prediction

In [ ]:
# Segunda Imagem
test_image = image.load_img('dataset_teste/2.jpg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = classifier.predict(test_image)
training_set.class_indices

if result[0][0] == 1:
    prediction = 'Cor 1'
if result[0][0] == 2:
    prediction = 'Cor 2'
if result[0][0] == 3:
    prediction = 'Cor 3'
if result[0][0] == 4:
    prediction = 'Cor 4'
else:
    prediction = 'Cor 5'

Image(filename='dataset_teste/2.jpg')

In [ ]:
# Previsão da segunda imagem
prediction

Nosso modelo acertou mais uma! Veja que a imagem possui apenas a face de um gato, mas os detalhes estão bem definidos e nosso modelo não teve trabalho nesta classificação.

In [ ]:
# Terceira Imagem
test_image = image.load_img('dataset_teste/3.jpg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = classifier.predict(test_image)
training_set.class_indices

if result[0][0] == 1:
    prediction = 'Cor 1'
if result[0][0] == 2:
    prediction = 'Cor 2'
if result[0][0] == 3:
    prediction = 'Cor 3'
if result[0][0] == 4:
    prediction = 'Cor 4'
else:
    prediction = 'Cor 5'

Image(filename='dataset_teste/3.jpg')

In [ ]:
# Previsão da terceira imagem
prediction

Nosso modelo está impossível. Acertou novamente! :-)

In [ ]:
# Quarta Imagem
test_image = image.load_img('dataset_teste/4.jpg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = classifier.predict(test_image)
training_set.class_indices

if result[0][0] == 1:
    prediction = 'Cor 1'
if result[0][0] == 2:
    prediction = 'Cor 2'
if result[0][0] == 3:
    prediction = 'Cor 3'
if result[0][0] == 4:
    prediction = 'Cor 4'
else:
    prediction = 'Cor 5'
    
Image(filename='dataset_teste/4.jpg')

In [ ]:
# Previsão da quarta imagem
prediction

Veja que esse modelo está realmente bem treinado, pois ele é capaz de classificar corretamente apenas a face de um gato ou mesmo a foto do corpo inteiro de um cachorro! Para que isso aconteça, temos que apresentar muitas imagens ao modelo durante o treinamento.

In [ ]:
# Quinta Imagem
test_image = image.load_img('dataset_teste/5.jpg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = classifier.predict(test_image)
training_set.class_indices
if result[0][0] == 1:
    prediction = 'Cor 1'
if result[0][0] == 2:
    prediction = 'Cor 2'
if result[0][0] == 3:
    prediction = 'Cor 3'
if result[0][0] == 4:
    prediction = 'Cor 4'
else:
    prediction = 'Cor 5'

Image(filename='dataset_teste/5.jpg')

In [ ]:
# Previsão da quinta imagem
prediction

Ops. Nosso modelo falhou! Por que isso ocorreu? Nosso modelo não conseguiu detectar na imagem as características aprendidas sobre gatos e classificou como cachorro. Para resolver isso, teríamos que retornar ao treinamento e apresentar mais imagens ao modelo, eventualmente usando técnicas como Dataset Augmentation e treinar por mais tempo, até atingir uma acurácia maior. Ensinamos tudo isso aos alunos nos cursos da Formação Inteligência Artificial, aqui na DSA.

In [ ]:
# Sexta Imagem
test_image = image.load_img('dataset_teste/5.jpg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = classifier.predict(test_image)
training_set.class_indices
if result[0][0] == 1:
    prediction = 'Cor 1'
if result[0][0] == 2:
    prediction = 'Cor 2'
if result[0][0] == 3:
    prediction = 'Cor 3'
if result[0][0] == 4:
    prediction = 'Cor 4'
else:
    prediction = 'Cor 5'

Image(filename='dataset_teste/5.jpg')

In [ ]:
# Previsão da sexta imagem
prediction

## Conclusão

Obteve-se um resultado final de 95,82% de precisão para o nosso conjunto de treino e 83% para o nosso conjunto de testes. Para nossas 6 imagens de amostra, nosso modelo acertou 5. Obviamente mais testes poderiam ser realizados. 

Melhorias adicionais para este modelo:

- Aumentar o número de épocas para 25 para uma aprendizagem mais profunda. 
- Além disso, aumentar o redimensionamento da imagem de 64x64 para 256x256 deve levar a melhores resultados devido à resolução mais alta. 
- Aumentar o tamanho do lote de 32 para 64 também pode levar a melhores resultados.
- Usar imagens sintéticas rotacionando a imagem principal, técnica conhecida como Dataset Augmentation.
- Alterar a arquitetura da rede incluindo mais uma camada convolucional.
- Avaliar outras métricas do modelo e ajustar os hiperparâmetros de acordo.